In [6]:
import robustsp as rsp
import numpy as np

def spec_arma_est_bip_tau(x,p,q):
    x = x - np.median(x)
    
    N = len(x)
    
    w = np.linspace(0,np.pi,int(N/2))
    # Digital frequency must be used for this calculation
    s = np.exp(1j*w)
    
    result = rsp.arma_est_bip_tau(x,p,q)
    
    beta_hat = np.zeros(p+q)
    beta_hat[:p] = result['ar_coeffs']
    beta_hat[p:] = result['ma_coeffs']
    
    Xx = np.polyval([1,*beta_hat[p:]],s)\
    / np.polyval([1,*beta_hat[:p]],s)
    
    sigma_hat = result['inno_scale']
    # BIP-ARMA tau spectral estimate 
    Pxx = (sigma_hat**2)/(2*np.pi) *np.abs(Xx)**2
    
    PxxdB = 10 *np.log10(Pxx)
    
    return PxxdB, Pxx, w, sigma_hat

In [7]:
import scipy.io as sio
ld = lambda x: sio.loadmat(x)[x].flatten()

y = ld('y')

PxxdB_taum = ld('PxxdB_tau')

Pxx_taum = ld('Pxx_tau')

wm = ld('w')

sigma_hatm = ld('sigma_hat')

p = 4

q = 3

In [8]:
PxxdB_tau, Pxx_tau, w, sigma_hat = spec_arma_est_bip_tau(y,p,q)

In [15]:
eq = lambda x,y: np.array_equal(x,y)
ic = lambda x,y: np.isclose(x,y,1e-1).all()
ic(PxxdB_tau, PxxdB_taum)

False

In [10]:
sigma_hat

1.1654267229534228

In [11]:
sigma_hatm

array([1.16860421])

In [12]:
ic(w, wm)

True

In [16]:
b_0 = 0.13137;
np.array([0.13137, 0.023543, 0.10775, 0.03516])/b_0

array([1.        , 0.17921139, 0.82020248, 0.26764101])

In [17]:
Pxx_tau

array([1.45321734e+00, 1.45420072e+00, 1.45715694e+00, 1.46210438e+00,
       1.46907398e+00, 1.47810962e+00, 1.48926879e+00, 1.50262341e+00,
       1.51826085e+00, 1.53628527e+00, 1.55681919e+00, 1.58000541e+00,
       1.60600932e+00, 1.63502167e+00, 1.66726183e+00, 1.70298175e+00,
       1.74247066e+00, 1.78606071e+00, 1.83413370e+00, 1.88712933e+00,
       1.94555498e+00, 2.00999781e+00, 2.08113942e+00, 2.15977391e+00,
       2.24683021e+00, 2.34339984e+00, 2.45077173e+00, 2.57047616e+00,
       2.70434062e+00, 2.85456162e+00, 3.02379758e+00, 3.21529041e+00,
       3.43302633e+00, 3.68195098e+00, 3.96826086e+00, 4.29980371e+00,
       4.68663666e+00, 5.14181742e+00, 5.68254587e+00, 6.33184431e+00,
       7.12108536e+00, 8.09388823e+00, 9.31228864e+00, 1.08668066e+01,
       1.28934328e+01, 1.56033638e+01, 1.93371786e+01, 2.46677129e+01,
       3.26027757e+01, 4.49907076e+01, 6.52758401e+01, 9.92830372e+01,
       1.51262191e+02, 1.97335244e+02, 1.79261697e+02, 1.20682345e+02,
      

In [20]:
np.sum(np.abs((Pxx_taum-Pxx_tau))**2)

7269.457313717727

# ARMA est bip tau

inno_scale mit robust_starting_point von matlab richtig

In [1]:
import robustsp as rsp
import numpy as np
import scipy.io as sio
ld = lambda x: sio.loadmat(x)[x].flatten()

y = ld('y')

p = 4

q = 3

meth= None
from scipy.optimize import minimize
y = y -np.median(y)

In [2]:
'''
beta_initial =
1.3678   -1.5214    1.1391   -0.8203   -0.0230   -0.8493   -0.0945
'''
# Robust starting point by BIP AR-tau approximation
beta_initial = rsp.robust_starting_point(y,p,q)[0]
beta_initial

array([ 1.35251665, -1.50851952,  1.1431666 , -0.83287937, -0.03934287,
       -0.8588762 , -0.0817226 ])

In [13]:
# from matlab
beta_initial = \
np.array([1.3678   ,-1.5214    ,1.1391  , -0.8203   ,-0.0230  , -0.8493,-0.0945])

In [3]:
F = lambda beta: rsp.arma_tau_resid_sc(y,beta,p,q)

F_bip = lambda beta: rsp.bip_tau_resid_sc(y,beta,p,q)[0]

In [4]:
beta_arma_m =\
np.array([1.4270 ,  -1.5565    ,1.0681  , -0.7705   ,-0.1118 ,  -0.6810 ,   0.0016])
beta_arma = minimize(F, beta_initial, method='SLSQP')['x']
# F(beta_arma) should be 10e10, is 10e10
# F(beta_arma_m) should be 3.8418 is correct
F(beta_arma), F(beta_arma_m)

(10000000000, 3.8418493526882886)

In [10]:
beta_arma

array([-0.18758367, -0.19099155, -0.21314403,  0.93938803, -1.19338321,
       -0.72645468,  0.19016781])

In [5]:
beta_bip_m=np.array([1.3168,-1.4657,1.1267,-0.8317,-0.0967,-0.8777,-0.1608])
beta_bip = minimize(F_bip, beta_initial, method='SLSQP')['x']
F_bip(beta_bip), F_bip(beta_bip_m) # should be 1.1686

(1.1654267229534228, 1.1686097801345536)

In [17]:
# final parameter estimate uses the model that provides smallest tau_scale
beta_hat = beta_arma if a_sc<a_bip_sc else beta_bip

# final tau scale
a_tau_sc = min(a_sc, a_bip_sc)

# Output the results...

NameError: name 'a_sc' is not defined

In [18]:
# -1.3168 1.4657 -1.1267 0.8317
-beta_hat[:p]

NameError: name 'beta_hat' is not defined

In [ ]:
# 0.0967 0.8777 0.1608
-beta_hat[p:]

In [ ]:
#1.1686
a_tau_sc